In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn import datasets

In [13]:
clusters = {}
centroids = []
np_data = []
total_for_coreset = 0
#function to generate 'num_points' random points of 'dim' dimensions.
def generate_data(data, m):
	#if data_type == 1:
	#	pass
	#filename = sys.argv[1] #dataset to calculate coreset of
	#output = sys.argv[2] #output file to print probability distribution values
	dim = data.shape[1]
	rows = data.shape[0]
	data_mean = np.mean(data, axis = 0)
	#print(data_mean)
	distance = 0
	for point in data:
		distance += np.sum(np.square(point-data_mean))
	#print(distance)
	prob_dist = []
	#calculating proposal  distribution for each row
	for point in data:
		value=((0.5*(1/n))+0.5*((np.sum(np.square(point-data_mean)))/distance))
		prob_dist.append(value)
	df = pd.DataFrame(prob_dist)
	data = pd.DataFrame(data)
	data['Prob_dist'] = df
	weight_value = []
	for i in range(n):
		weight = 1/(m*prob_dist[i])
		weight_value.append(weight)
	df = pd.DataFrame(weight_value)
	data['weight_value'] = df
	sampled_points = data.sample(n=m, frac=None, replace=False, weights='weight_value', random_state=None, axis=None)
	return sampled_points.iloc[:,:dim]

In [14]:
# Loading data
data = datasets.fetch_covtype()

# Identifying the number of classes
k = 7
data = pd.DataFrame(data.data)

data = np.array(data)

# Identifying data length
n = data.shape[0]

## Finding the number of attributes.
features = data.shape[1]


start = timeit.default_timer()
points = np.array(generate_data(data, k))
stop = timeit.default_timer()
total_for_coreset = stop-start

## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k, init = points, n_init =1).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start
# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## generating Query point
query = []
query = np.random.randint(low=0, high=3000, size=features)
query = query.reshape(1,features)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
stop = timeit.default_timer()   
time_for_closest_center = stop-start   
## Finding the cluster elements to construct KD-Tree

cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(total_for_coreset,time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(total_for_coreset + time_for_kmeans + time_for_closest_center + time_for_kdtree + query_time)

[[2.347e+03 3.190e+02 1.000e+01 4.840e+02 2.720e+02 1.482e+03 1.930e+02
  2.310e+02 1.770e+02 1.041e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.338e+03 3.150e+02 1.400e+01 4.810e+02 2.570e+02 1.465e+03 1.820e+02
  2.280e+02 1.850e+02 1.036e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00

In [15]:
start = timeit.default_timer()
tree = KDTree(data)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
j=0
start = timeit.default_timer()
dist_dum , ind_dum = tree.query(query,n_n)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
for i in ind_dum:
    print(data[i],dist_dum[j])
    j =+ 1

2.8154542500005846
0.0018837709994841134
[[2.347e+03 3.190e+02 1.000e+01 4.840e+02 2.720e+02 1.482e+03 1.930e+02
  2.310e+02 1.770e+02 1.041e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.338e+03 3.150e+02 1.400e+01 4.810e+02 2.570e+02 1.465e+03 1.820e+02
  2.280e+02 1.850e+02 1.036e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 

In [29]:
# Loading data
data = datasets.fetch_covtype()

# Identifying the number of classes
k = 7
data = pd.DataFrame(data.data)

data = np.array(data)

# Identifying data length
n = data.shape[0]

## Finding the number of attributes.
features = data.shape[1]

start = timeit.default_timer()
## Running KMeans on train data.
y_predict = KMeans(n_clusters = k).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start

# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
    
stop = timeit.default_timer()   
time_for_closest_center = stop-start    
## Finding the cluster elements to construct KD-Tree
cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start

j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1

print(time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(time_for_closest_center+time_for_kdtree+query_time)

[[2.347e+03 3.190e+02 1.000e+01 4.840e+02 2.720e+02 1.482e+03 1.930e+02
  2.310e+02 1.770e+02 1.041e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.338e+03 3.150e+02 1.400e+01 4.810e+02 2.570e+02 1.465e+03 1.820e+02
  2.280e+02 1.850e+02 1.036e+03 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00

In [30]:
cluster_values.shape

(159981, 54)

In [31]:
data.shape

(581012, 54)